In [51]:
from collections import defaultdict
import pandas as pd
import math
df = pd.read_csv("csi_failure_dataset_open_source_systems.csv")
df = df.fillna('')

## Section 4: Table 1

In [61]:
upstream_systems = list(df['Upstream System'])
downstream_systems = list(df['Downstream System'])
dic = defaultdict(int)
for systems in list(zip(upstream_systems, downstream_systems)):
    dic[systems[0] + "_" + systems[1]] += 1
system_order = [["Spark", "Hive"], ["Spark", "YARN"], ["Spark", "HDFS"], ["Spark", "Kafka"],
                ["Flink", "Kafka"], ["Flink", "YARN"], ["Flink", "Hive"], ["Flink", "HDFS"],
               ["Hive", "Spark"], ["Hive", "HBase"], ["Hive", "HDFS"], ["Hive", "Kafka"], ["Hive", "YARN"],
               ["HBase", "HDFS"], ["YARN", "HDFS"]]
upstream_sys_col = list()
downstream_sys_col = list()
freq_col = list()
for systems in system_order:
    upstream_sys_col.append(systems[0])
    downstream_sys_col.append(systems[1])
    freq_col.append(dic[systems[0] + "_" + systems[1]])
table = pd.DataFrame([upstream_sys_col, downstream_sys_col, freq_col]).transpose()
table.columns = ['Upstream', 'Downstream','# CSI Failures']
table

,Upstream,Downstream,# CSI Failures
0,Spark,Hive,26
1,Spark,YARN,19
2,Spark,HDFS,8
3,Spark,Kafka,5
4,Flink,Kafka,12
5,Flink,YARN,14
6,Flink,Hive,8
7,Flink,HDFS,3
8,Hive,Spark,6
9,Hive,HBase,3


## Section 5: Table 2

In [63]:
issues =  list(df['Failure Plane'])
total_issues = len(issues)
count = pd.Series(issues).value_counts()
planes = ["Control", "Data", "Management"]
vals = [count[plane] for plane in cols]
percentages = [str(round(count[plane]/total_issues * 100, 2)) + "%" for plane in cols]
table = pd.DataFrame([planes, vals, percentages]).transpose()
table.columns = ['Plane', '# Failures','Percentage']
table

,Plane,# Failures,Percentage
0,Control,20,16.67%
1,Data,61,50.83%
2,Management,39,32.5%


## Section 5: Table 3

In [67]:
planes =  list(df['Failure Plane'])
consequences = list(df['Failure Consequence'])
plane_to_consequences = defaultdict(lambda: defaultdict(int))
for plane_and_conseq in list(zip(planes, consequences)):
    plane_to_consequences[plane_and_conseq[0]][plane_and_conseq[1].strip()] += 1
print(plane_to_consequences)
control_plane_rows = ["Runtime crash/hang", "Startup failure", "Performance issue", "Data loss", "Unexpected Behaviour"]
freq = list()
for consequence in control_plane_rows:
    freq.append(plane_to_consequences["Control"][consequence])
control_plane_table = pd.DataFrame([control_plane_rows, freq]).transpose()
control_plane_table.columns = ['Impact','#']


data_plane_rows = ["Job/task failure", "Job/task startup failure", "Wrong results", "Performance issue", "Resource leak", "Usability"]
freq = list()
for consequence in data_plane_rows:
    freq.append(plane_to_consequences["Data"][consequence])
data_plane_table = pd.DataFrame([data_plane_rows, freq]).transpose()
data_plane_table.columns = ['Impact','#']


management_plane_rows = ["Job/task crash/hang","Reduced observability", "Unexpected Behaviour", "Performance issue"]
freq = list()
for consequence in management_plane_rows:
    freq.append(plane_to_consequences["Management"][consequence])
management_plane_table = pd.DataFrame([management_plane_rows, freq]).transpose()
management_plane_table.columns = ['Impact','#']


defaultdict(<function <lambda> at 0x5483450>, {'Control': defaultdict(<class 'int'>, {'Runtime crash/hang': 8, 'Unexpected Behaviour': 3, 'Startup failure': 4, 'Performance issue': 3, 'Data loss': 2}), 'Data': defaultdict(<class 'int'>, {'Resource leak': 2, 'Job/task failure': 47, 'Performance issues': 2, 'Job/task startup failure': 6, 'Wrong results': 3, 'Usability': 1}), 'Management': defaultdict(<class 'int'>, {'Reduced observability': 8, 'Performance issue': 2, 'Job/task crash/hang': 24, 'Unexpected Behaviour': 5})})


In [68]:
control_plane_table

,Impact,#
0,Runtime crash/hang,8
1,Startup failure,4
2,Performance issue,3
3,Data loss,2
4,Unexpected Behaviour,3


In [69]:
data_plane_table

,Impact,#
0,Job/task failure,47
1,Job/task startup failure,6
2,Wrong results,3
3,Performance issue,0
4,Resource leak,2
5,Usability,1


In [70]:
management_plane_table

,Impact,#
0,Job/task crash/hang,24
1,Reduced observability,8
2,Unexpected Behaviour,5
3,Performance issue,2
